In [1]:
import json
import os
import functools
import math
import copy

import pprint
pp = pprint.PrettyPrinter(indent=4, depth=10, width=120)





In [ ]:
recipes = None
with open('/home/tuenut/Downloads/Telegram Desktop/recipes.json', 'r') as file:
    recipes = json.load(file, )

In [ ]:
pp.pprint(recipes)

In [ ]:
recipes['copper-plate'], recipes['copper-cable']

In [ ]:
PRODUCT_TYPES = {'advanced-crafting',
 'centrifuging',
 'chemistry',
 'crafting',
 'crafting-with-fluid',
 'oil-processing',
 'rocket-building',
 'smelting'}

In [ ]:
producers = {
    "electric-furnace": {
        "name": "electric-furnace", 
        "prod_types": ['smelting'], 
        "speed": 2,
        "mod slots": 2
    },
    "assembling-machine-1": {
        "name": "assembling-machine-1", 
        "prod_types": ['crafting'], 
        "speed": 0.5,
        "mod slots": 0
    },
    "assembling-machine-2": {
        "name": "assembling-machine-2", 
        "prod_types": ['crafting', 'crafting-with-fluid', 'advanced-crafting'], 
        "speed": 0.75,
        "mod slots": 2
    },
    "assembling-machine-3": {
        "name": "assembling-machine-3", 
        "prod_types": ['crafting', 'crafting-with-fluid', 'advanced-crafting'], 
        "speed": 1.25,
        "mod slots": 4
    },
    "oil-refinery.basic": {
        "name": "oil-refinery.basic", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "oil-refinery.advanced": {
        "name": "oil-refinery.advanced", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "oil-refinery.coal-liquefaction": {
        "name": "oil-refinery.coal-liquefaction", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "chemical-plant": {
        "name": "chemical-plant", 
        "prod_types": ['chemistry'], 
        "speed": 1.25,
        "mod slots": 3
    },
}

In [ ]:
class Calculator:
    product = None
    product_asms = None
    assemblers_schema = []
    
    EXCLUDE_LIST = ['iron-ore', 'copper-ore', 'coal', 'petroleum-gas', 'stone', 'water']
    
    def input_data(self, product_name=None, amount_per_minute=0, available_assemblers=[]):
        self.assemblers_schema = []
        self.product = recipes[product_name]
        self.product_asms = available_assemblers
        self.target_speed = amount_per_minute / 60

    def calc(self, product_name=None, amount=1, ):
        if not self.product_asms:
            return "Cant craft without assemblers. Self craft not implemented."
        
        product = self.product if not product_name else recipes[product_name]
                
        asm = self._get_asm(self.product['category'])
        product_time = product['energy'] / asm['speed'] * amount
#         if product_name:
#             product_time *= count
        product_speed = 1 / product_time 
        
        count = self.target_speed / product_speed
            
        for item in product['ingredients']:
            if item['name'] not in self.EXCLUDE_LIST:
                print(item['name'])
                self.calc(product_name=item['name'], amount=item['amount'], )

        self.assemblers_schema.append({
            product['name']: {
                'count of producer': count,
                'int count of producer': math.ceil(count),
                'producer': asm['name'],
                'speed': product_speed
            }
        })
        
#         if not product_name:
#             for item in product['ingredients']:
#                 self.assemblers_schema
        
        return self.assemblers_schema
                
    def _get_asm(self, prod_type):
        asm = [producers[asm_name] for asm_name in self.product_asms if prod_type in producers[asm_name]['prod_types']][0]
        return asm

In [ ]:
calculator = Calculator()

In [ ]:
calculator.input_data(product_name='assembling-machine-1', amount_per_minute=500, available_assemblers=['assembling-machine-3', 'electric-furnace'])

In [ ]:
calculator.calc()

In [ ]:
4*

In [ ]:
8.75*60

In [ ]:
5/2

In [78]:
"""
Класс продукта:
    - рецепт dict
    - базовый выход dict
    - @property выход
        вернуть базовый выход, если вход не указан или рассчитать на основе входа
    - метод сложения двух продуктов.
        на выходе получаем продукт с суммой базовых выходов и рецептов обоих продуктов
        продуктов относительно входа или вернут 1 для каждого, если вход не указан.
        
        
"""

class Product:
    recipe = dict()
    base_output = dict()
    _input = dict()
    
    def __init__(self, recipe=None, base_output=None, raw=None):
        self.recipe = recipe if recipe else self.recipe
        self.base_output = base_output if base_output else self.base_output
        self.raw = raw if raw else None
    
    def __add__(self, other):
        klass = Product()
        
        klass.recipe = copy.deepcopy(self.recipe)
        for key, val in other.recipe.items():
            try:
                klass.recipe[key] += val
            except KeyError:
                klass.recipe[key] = val
                
        klass.base_output = copy.deepcopy(self.base_output)
        for key, val in other.base_output.items():            
            try:
                klass.base_output[key] += val
            except KeyError:
                klass.base_output[key] = val
                
        raw = copy.deepcopy(self.raw)
        for key, val in other.raw.items():
            try:
                raw[key] += val
            except KeyError:
                raw[key] = val
        klass.raw = raw
        
        return klass
    
    def __mul__(self, other):
        klass = Product(recipe=copy.deepcopy(self.recipe),
                        base_output=copy.deepcopy(self.base_output))
        for key, val in klass.recipe.items():
            try:
                klass.recipe[key] *= other
            except KeyError:
                klass.recipe[key] = other
                
        for key, val in klass.base_output.items():            
            try:
                klass.base_output[key] *= other
            except KeyError:
                klass.base_output[key] = other
                
        raw = copy.deepcopy(self.raw)
        for key, val in klass.raw.items():
            try:
                raw[key] *= other
            except KeyError:
                raw[key] = other
        klass.raw = raw
        
        return klass
    
    def __str__(self):
        return 'In {1}\nOut {0}'.format(str(self.base_output), str(self.recipe))
    
    __repr__ = __str__
    
    @property
    def raw(self, ):
        return self._input if self._input else self.recipe
    
    @raw.setter
    def raw(self, value):
        if isinstance(value, int):
            self._input = {key: value for key in self.recipe.items()}
        elif isinstance(value, dict):
            self._input = copy.deepcopy(value)
        else:
            return False
    
    @property
    def produce(self, ):        
        factors = {}
        for key, val in self.recipe.items():
            factors[key] = self.raw[key] / val if key in self.raw else 0
        factor = min(factors.values())
       
        vanila_prod = {key: val*factor for key, val in self.base_output.items()}
        
        for key, val in vanila_prod.items():
            if key in self.recipe:
                vanila_prod[key] -= self.recipe[key] * factor
        return vanila_prod
            
        
basic = Product(recipe={'oil': 100}, base_output={'heavy': 30, 'light': 30, 'petroleum': 40})
advanced = Product(recipe={'oil': 100}, base_output={'heavy': 10, 'light': 45, 'petroleum': 55})
heavy = Product(recipe={'heavy': 40}, base_output={'light': 30})
light = Product(recipe={'light': 30}, base_output={'petroleum': 20})



In [109]:
c = advanced*10 + basic*4 + heavy*5 + light*24
c.produce, c.raw

({'heavy': 20.0, 'light': 0.0, 'petroleum': 1190.0},
 {'oil': 1400, 'heavy': 200, 'light': 720})

In [104]:
c = advanced*14 + heavy*3 + light*24
c.produce, c.raw

({'heavy': 20.0, 'light': 0.0, 'petroleum': 1250.0},
 {'oil': 1400, 'heavy': 120, 'light': 720})

({'heavy': 20.0, 'light': 0.0, 'petroleum': 1250.0},
 {'oil': 1400, 'heavy': 120, 'light': 720})

In [74]:
c = advanced*1
c.raw, c.recipe, c.produce

{'oil': 1.0} 1.0


({'oil': 100}, {'oil': 100}, {'heavy': 10.0, 'light': 45.0, 'petroleum': 55.0})

In [ ]:
a.symmetric_difference(b)

In [ ]:
min({'a':1, 'b':2})